In [29]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from keras.wrappers.scikit_learn import KerasClassifier

In [30]:
headers = ['name', 'hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize', 'type']

In [31]:
df = pd.read_csv('zoo.data', header=None)

In [32]:
df.columns = headers

In [33]:
df.head()

,name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [34]:
arr = np.array(df)

In [35]:
X_with_name = arr[:, :-1]

In [36]:
X = X_with_name[:, 1:].astype('int')

In [37]:
Y = arr[:, -1].astype('int')

In [38]:
models = [
    SVC(kernel='linear'),
    SVC(kernel='rbf'),
    SVC(kernel='poly'),
    SVC(kernel='sigmoid')
]

In [39]:
cross = KFold(5)

In [40]:
for model in models:
    best_score = -1
    for train, test in cross.split(X, Y):
        X_train = X[train]
        Y_train = Y[train]
        
        X_test = X[test]
        Y_test = Y[test]
        
        trained = model.fit(X_train, Y_train)
        best_score = max(best_score, trained.score(X_test, Y_test))
    print("{} - {}".format(model.kernel, best_score))

linear - 1.0
rbf - 0.9523809523809523
poly - 1.0
sigmoid - 0.8095238095238095


/Users/jatinkatyal/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/jatinkatyal/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/jatinkatyal/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/jatinkatyal/miniconda3/lib/

In [41]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import keras_metrics

In [42]:
model = Sequential()
model.add(Dense(12, input_dim=16, kernel_initializer='uniform', activation='relu'))
model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
best_score = -1
for train, test in cross.split(X, Y):
    train, test = list(train), list(test)
    X_train = X[train]
    Y_train = Y[train]

    X_test = X[test]
    Y_test = Y[test]
    
    model.fit(X_train, to_categorical(Y_train), epochs=75, verbose=False)
    best_score = max(best_score, model.evaluate(X_test, to_categorical(Y_test, num_classes=8))[-1])

20/20 [==============================] - 0s 20us/step


In [44]:
best_score

0.9624999761581421

In [59]:
y_pred = model.predict(X_test).ravel()